In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

#pd.set_option('display.max_rows', None)
pd.reset_option("display.min_rows")
#-----------------------------------------------------------
#Lê os dados do arquivo CSV.
#-----------------------------------------------------------
data = pd.read_csv('Inteligencia_Artificial/Trabalho de Conclusão de Curso/Base de Dados/2021/BPAC_30m_2021.csv') 
data.head()
#print(df)
#df.info()

,time,open,high,low,close,Volume,Volume MA
0,2021-01-04T10:00:00-03:00,23.7500,23.8750,23.6125,23.6225,401600,492360
1,2021-01-04T10:30:00-03:00,23.6300,23.6725,23.4225,23.5300,358000,476720
2,2021-01-04T11:00:00-03:00,23.5225,23.5375,23.4475,23.5000,258800,455300
3,2021-01-04T11:30:00-03:00,23.5000,23.5200,23.4300,23.4825,273200,436000
4,2021-01-04T12:00:00-03:00,23.4875,23.5125,23.3875,23.3900,762000,457560


In [ ]:
data = data.set_index('time') #Mudando o index para o date
#print(df.index)

#-----------------------------------------------------------
#Sabe se mudou o index, porque com o df.info, ele não aparece como um dos parametros
#-----------------------------------------------------------
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3932 entries, 2021-01-04T10:00:00-03:00 to 2021-12-30T17:30:00-03:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   open       3932 non-null   float64
 1   high       3932 non-null   float64
 2   low        3932 non-null   float64
 3   close      3932 non-null   float64
 4   Volume     3932 non-null   int64  
 5   Volume MA  3932 non-null   int64  
dtypes: float64(4), int64(2)
memory usage: 215.0+ KB


In [ ]:
#-----------------------------------------------------------
#Escolhe a coluna de série temporal única de interesse.
#-----------------------------------------------------------
symbol = "close"

data = pd.DataFrame(data[symbol])
data.head()

,close
time,
2021-01-04T10:00:00-03:00,23.6225
2021-01-04T10:30:00-03:00,23.5300
2021-01-04T11:00:00-03:00,23.5000
2021-01-04T11:30:00-03:00,23.4825
2021-01-04T12:00:00-03:00,23.3900


In [ ]:
#-----------------------------------------------------------
#Renomeia a única coluna para price
#-----------------------------------------------------------

data.rename(columns={symbol: 'price'}, inplace=True)

data.head()

,price
time,
2021-01-04T10:00:00-03:00,23.6225
2021-01-04T10:30:00-03:00,23.5300
2021-01-04T11:00:00-03:00,23.5000
2021-01-04T11:30:00-03:00,23.4825
2021-01-04T12:00:00-03:00,23.3900


In [ ]:
#-----------------------------------------------------------
# Calcula o log de retorno e define a direção como uma coluna binária.
#-----------------------------------------------------------

#Log Retorno
data['return'] = np.log(data['price']/data['price'].shift(1))


#Coluna binária
data['direction'] = np.where(data['return'] > 0, 1, 0)
#data.head()

display(data)

,price,return,direction
time,,,
2021-01-04T10:00:00-03:00,23.6225,NaN,0
2021-01-04T10:30:00-03:00,23.5300,-0.003923,0
2021-01-04T11:00:00-03:00,23.5000,-0.001276,0
2021-01-04T11:30:00-03:00,23.4825,-0.000745,0
2021-01-04T12:00:00-03:00,23.3900,-0.003947,0
...,...,...,...
2021-12-30T15:30:00-03:00,21.0300,0.001428,1
2021-12-30T16:00:00-03:00,21.0200,-0.000476,0
2021-12-30T16:30:00-03:00,20.9800,-0.001905,0


In [ ]:
#Para a previsão do mercado de ações, é crucial encontrar dependências de tempo bem definidas
lags = 4

cols = []
#-----------------------------------------------------------
# Cria os Lagged Data
#-----------------------------------------------------------
for lag in range(1, lags + 1):
     col = f'lag_{lag}'
#-----------------------------------------------------------
# Cria novas colunas DataFrame com os retornos de log deslocados pelo respectivo número de atrasos
#-----------------------------------------------------------
     data[col] = data['return'].shift(lag)
     cols.append(col)
#-----------------------------------------------------------
# Exclui as linhas que contêm valores NaN.
#-----------------------------------------------------------
data.dropna(inplace=True)
data[col]

time
2021-01-04T12:30:00-03:00   -0.003923
2021-01-04T13:00:00-03:00   -0.001276
2021-01-04T13:30:00-03:00   -0.000745
2021-01-04T14:00:00-03:00   -0.003947
2021-01-04T14:30:00-03:00   -0.006003
                               ...   
2021-12-30T15:30:00-03:00    0.007656
2021-12-30T16:00:00-03:00    0.000000
2021-12-30T16:30:00-03:00    0.000000
2021-12-30T17:00:00-03:00    0.000953
2021-12-30T17:30:00-03:00    0.001428
Name: lag_4, Length: 3927, dtype: float64

In [ ]:
#-----------------------------------------------------------
# Imprime as cinco linhas finais indicando os “padrões” emergentes nas quatro colunas de recursos
#-----------------------------------------------------------

#data.round(4).tail() 
display(data.index)

Index(['2021-01-04T12:30:00-03:00', '2021-01-04T13:00:00-03:00',
       '2021-01-04T13:30:00-03:00', '2021-01-04T14:00:00-03:00',
       '2021-01-04T14:30:00-03:00', '2021-01-04T15:00:00-03:00',
       '2021-01-04T15:30:00-03:00', '2021-01-04T16:00:00-03:00',
       '2021-01-04T16:30:00-03:00', '2021-01-04T17:00:00-03:00',
       ...
       '2021-12-30T13:00:00-03:00', '2021-12-30T13:30:00-03:00',
       '2021-12-30T14:00:00-03:00', '2021-12-30T14:30:00-03:00',
       '2021-12-30T15:00:00-03:00', '2021-12-30T15:30:00-03:00',
       '2021-12-30T16:00:00-03:00', '2021-12-30T16:30:00-03:00',
       '2021-12-30T17:00:00-03:00', '2021-12-30T17:30:00-03:00'],
      dtype='object', name='time', length=3927)

In [ ]:
#-----------------------------------------------------------
# Define a data limite entre os dados de treinamento e teste.
#-----------------------------------------------------------
data.index = range(len(data['direction']))
cutoff = int(len(data['direction']) * 0.80)
print(cutoff)


display(data)

3141


,price,return,direction,lag_1,lag_2,lag_3,lag_4
0,23.250,-0.006003,0,-0.003947,-0.000745,-0.001276,-0.003923
1,23.190,-0.002584,0,-0.006003,-0.003947,-0.000745,-0.001276
2,23.100,-0.003889,0,-0.002584,-0.006003,-0.003947,-0.000745
3,23.185,0.003673,1,-0.003889,-0.002584,-0.006003,-0.003947
4,23.225,0.001724,1,0.003673,-0.003889,-0.002584,-0.006003
...,...,...,...,...,...,...,...
3922,21.030,0.001428,1,0.000953,0.000000,0.000000,0.007656
3923,21.020,-0.000476,0,0.001428,0.000953,0.000000,0.000000
3924,20.980,-0.001905,0,-0.000476,0.001428,0.000953,0.000000
3925,20.900,-0.003820,0,-0.001905,-0.000476,0.001428,0.000953


In [ ]:
"""                      TREINAMENTO                      """
#-----------------------------------------------------------
# Define os conjuntos de dados de treinamento.
#-----------------------------------------------------------
training_data = data[data.index < cutoff].copy()


#-----------------------------------------------------------
# Normalizes the features data by Gaussian normalization. (z-score)
#-----------------------------------------------------------
mu, std = training_data.mean(), training_data.std()
training_data_ = (training_data - mu) / std
#print(training_data_[cols])



"""-------------------Dados de Entrada--------------------"""

#-----------------------------------------------------------
# Pega os dados de 'Lag' de treinamentos e coloca na variavel X_train
#-----------------------------------------------------------
X_train = training_data[cols].values
X_train_ = training_data_[cols].values


#-----------------------------------------------------------
# Redefine os dados de treinamento para serem 3D (TRAINING_DAYS, TimeSteps, FEATURES)
#-----------------------------------------------------------
X_train = X_train.reshape(len(X_train),lags,1)
X_train_ = X_train_.reshape(len(X_train_),lags,1)


"""-------------------Dados de Saida----------------------"""

#-----------------------------------------------------------
# Pega os dados de direção(0 e 1) de treinamento e coloca na variavel Y_train
#-----------------------------------------------------------
Y_train = training_data['direction'].values

X_train_.shape, Y_train.shape
#X_train

((3141, 4, 1), (3141,))

In [ ]:
#-----------------------------------------------------------
# Quantos 0 e quantos 1 possuem no meu dado de Treino? 
#-----------------------------------------------------------
count1 = 0
count0 = 0
for i in Y_train:
    if i == 1:
        count1 += 1
    elif i == 0:
        count0 += 1
print("TREINAMENTO ")
print("Quantos 1 tem na saida? ",count1)
print("Quantos 0 tem na saida? ",count0)
print("Total: ",len(Y_train))

TREINAMENTO 
Quantos 1 tem na saida?  1543
Quantos 0 tem na saida?  1598
Total:  3141


In [ ]:
"""                           TESTE                     """
#-----------------------------------------------------------
# Define os conjuntos de dados de teste.
#-----------------------------------------------------------
test_data = data[data.index >= cutoff].copy()
#print(test_data[cols])

#-----------------------------------------------------------
# Normalizes the features data by Gaussian normalization.
#-----------------------------------------------------------
test_data_ = (test_data - mu) / std
#print(test_data_[cols])

"""-------------------Dados de Entrada--------------------"""

#-----------------------------------------------------------
# Pega os dados de 'Lag' de teste e coloca na variavel X_test
#-----------------------------------------------------------
X_test = test_data[cols].values
X_test_ = test_data_[cols].values

#-----------------------------------------------------------
# Redefine os dados de 'Lag' teste para serem 3D (TRAINING_DAYS, TimeSteps, FEATURES)
#-----------------------------------------------------------
X_test = X_test.reshape(len(X_test),lags, 1)
X_test_ = X_test_.reshape(len(X_test_),lags, 1)


"""-------------------Dados de Saida----------------------"""

#-----------------------------------------------------------
# Pega os dados de direção(0 e 1) de teste e coloca na variavel Y_teste
#-----------------------------------------------------------
Y_test = test_data['direction'].values
print(X_test_.shape, Y_test.shape)


(786, 4, 1) (786,)


In [ ]:
#-----------------------------------------------------------
# Quantos 0 e quantos 1 possuem no meu dado de Teste? 
#-----------------------------------------------------------
count1 = 0
count0 = 0
for i in Y_test:
    if i == 1:
        count1 += 1
    elif i == 0:
        count0 += 1
        
print("TESTE ")
print("Quantos 1 tem na saida? ",count1)
print("Quantos 0 tem na saida? ",count0)
print("Total: ",len(Y_test))

TESTE 
Quantos 1 tem na saida?  365
Quantos 0 tem na saida?  421
Total:  786


In [ ]:
#-----------------------------------------------------------
# Importa o pacote TensorFlow.
#-----------------------------------------------------------
import tensorflow as tf
#-----------------------------------------------------------
# Importa o objeto de modelo necessário do Keras
#-----------------------------------------------------------
from keras.models import Sequential
#-----------------------------------------------------------
# Importa o objeto de camada relevante do Keras
#-----------------------------------------------------------
from keras.layers import Dense, Flatten,Dropout, LSTM, Embedding, Input

# Importa Layer weight regularizers
# https://keras.io/api/layers/regularizers/
#-----------------------------------------------------------
from tensorflow import keras

from keras import layers

from tensorflow.keras.optimizers import Adam, RMSprop
#-----------------------------------------------------------
# Otimizador https://keras.io/api/optimizers/
#-----------------------------------------------------------
optimizer = Adam(learning_rate=0.001)

In [ ]:
#-----------------------------------------------------------
# Um modelo Sequencial é instanciado.
#-----------------------------------------------------------
model=Sequential()
model.add(LSTM(50 ,activation = 'relu',input_shape=(lags,1)))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
#-----------------------------------------------------------
# Compila o objeto de modelo Sequencial para classificação.
#-----------------------------------------------------------
model.compile(optimizer= optimizer,
 loss='binary_crossentropy',
# https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
# https://keras.io/api/metrics/
metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 50)                10400     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%%time
#-----------------------------------------------------------
# Ajusta o modelo ao conjunto de dados de treinamento.
#-----------------------------------------------------------
model.fit(X_train_, # Lags
    Y_train, # 0 e 1
   epochs=50, batch_size=100, verbose=True, #Gera conjunto de dados de validação aleatórios (https://curiousily.com/posts/time-series-forecasting-with-lstms-using-tensorflow-2-and-keras-in-python/) # https://github.com/keras-team/keras/issues/597
          shuffle=False)

Epoch 1/50
32/32 [==============================] - 3s 6ms/step - loss: 0.6933 - accuracy: 0.5014
Epoch 2/50
32/32 [==============================] - 0s 6ms/step - loss: 0.6929 - accuracy: 0.5078
Epoch 3/50
32/32 [==============================] - 0s 6ms/step - loss: 0.6928 - accuracy: 0.5088
Epoch 4/50
32/32 [==============================] - 0s 7ms/step - loss: 0.6928 - accuracy: 0.5088
Epoch 5/50
32/32 [==============================] - 0s 7ms/step - loss: 0.6927 - accuracy: 0.5091
Epoch 6/50
32/32 [==============================] - 0s 7ms/step - loss: 0.6926 - accuracy: 0.5094
Epoch 7/50
32/32 [==============================] - 0s 7ms/step - loss: 0.6926 - accuracy: 0.5094
Epoch 8/50
32/32 [==============================] - 0s 6ms/step - loss: 0.6925 - accuracy: 0.5072
Epoch 9/50
32/32 [==============================] - 0s 6ms/step - loss: 0.6924 - accuracy: 0.5097
Epoch 10/50
32/32 [==============================] - 0s 6ms/step - loss: 0.6924 - accuracy: 0.5097
Epoch 11/50
32/32 [

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test_, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))



Accuracy: 51.53%
